Tutorial on buildin simple Agent with LangChain. Based on:

https://python.langchain.com/docs/use_cases/chatbots/

In [5]:
## Uncomment below to instal newest versiono f langchain 
# !pip install --upgrade --quiet  langchain langchain-openai

import os 

# Define API key for OPenAI
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "")


# An agent is just a chatbot that can use tools.

Tools being code that allows the LLM to interact with systems outside of its session. You can use pre-defined tools or define custom ones. There are also different agent types which have different purposes.

https://python.langchain.com/docs/integrations/tools/ <br>
https://python.langchain.com/docs/modules/agents/tools/custom_tools <br>
https://python.langchain.com/docs/modules/agents/agent_types/<br>
https://react-lm.github.io/


Lets start with a simple agent that can use a tool for Shell command sand bash scripting.The LLM can use it to execute any shell commands. A common use case for this is letting the LLM interact with your local file system.

Note: Shell tool does not work with Windows OS.

In [6]:
# Import tool 
from langchain.tools import ShellTool

shell_tool = ShellTool()

# Print tool description 
shell_tool.name
shell_tool.description
shell_tool.args

{'commands': {'title': 'Commands',
  'description': 'List of shell commands to run. Deserialized using json.loads',
  'anyOf': [{'type': 'string'},
   {'type': 'array', 'items': {'type': 'string'}}]}}

In [7]:
## Example usage of shell tool
print(shell_tool.run({"commands": ["echo 'Hello World!'", "pwd"]}))


Executing command:
 ["echo 'Hello World!'", 'pwd']
Hello World!
/media/nando/New_Volume/ChatGPT_things/AI_Agent_CSM/chatbot_tutorial



/home/nando/anaconda3/envs/huggingface/lib/python3.9/site-packages/langchain_community/tools/shell/tool.py:32: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


In [8]:
## Load default OpenAI chatbot
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor,create_react_agent # To load simple ReAct agent. Reason an act
from langchain import hub

## Define LLM
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)

# Define list of tools the LLM is going to use 
tools = [shell_tool]

# Get the template prompt to use - you can modify this!
prompt = hub.pull("hwchase17/react")

## Read the prompt template 
print(prompt)


input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'] template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}'


In [9]:
## Construct the ReAct agent by defining the llm, tools and prompt template
shell_Agent = create_react_agent(llm=llm,tools=tools,prompt=prompt)

# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=shell_Agent, tools=tools, verbose=True)

In [12]:
## Run the agent executor

input = 'What are the files in the current working directory?'
response = agent_executor.invoke({"input":str(input)})
response



> Entering new AgentExecutor chain...
I should use the terminal to list the files in the current working directory.
Action: terminal
Action Input: lsExecuting command:
 ls
ConversationalBot_Tutorial.ipynb
ethanol_test.data
SimpleAgent_Tutorial.ipynb
test.data
Tutorial_MosDEF.ipynb


/home/nando/anaconda3/envs/huggingface/lib/python3.9/site-packages/langchain_community/tools/shell/tool.py:32: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


I now know the final answer
Final Answer: The files in the current working directory are ConversationalBot_Tutorial.ipynb, ethanol_test.data, SimpleAgent_Tutorial.ipynb, test.data, and Tutorial_MosDEF.ipynb.

> Finished chain.


{'input': 'What are the files in the current working directory?',
 'output': 'The files in the current working directory are ConversationalBot_Tutorial.ipynb, ethanol_test.data, SimpleAgent_Tutorial.ipynb, test.data, and Tutorial_MosDEF.ipynb.'}

In [11]:
response['output']

'The files in the current working directory are ConversationalBot_Tutorial.ipynb, ethanol_test.data, SimpleAgent_Tutorial.ipynb, test.data, and Tutorial_MosDEF.ipynb.'